In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
import plotly.io
plotly.io.templates.default = 'plotly_white'
plotly.io.renderers.default = 'notebook_connected'

In [3]:
plotly.io.templates['plotly']

layout.Template({
    'data': {'bar': [{'error_x': {'color': '#2a3f5f'},
                      'error_y': {'color': '#2a3f5f'},
                      'marker': {'line': {'color': '#E5ECF6', 'width': 0.5}},
                      'type': 'bar'}],
             'barpolar': [{'marker': {'line': {'color': '#E5ECF6', 'width': 0.5}}, 'type': 'barpolar'}],
             'carpet': [{'aaxis': {'endlinecolor': '#2a3f5f',
                                   'gridcolor': 'white',
                                   'linecolor': 'white',
                                   'minorgridcolor': 'white',
                                   'startlinecolor': '#2a3f5f'},
                         'baxis': {'endlinecolor': '#2a3f5f',
                                   'gridcolor': 'white',
                                   'linecolor': 'white',
                                   'minorgridcolor': 'white',
                                   'startlinecolor': '#2a3f5f'},
                         'type': 'carpet'}],

In [4]:
DOCS_LINK = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQCN9pL21lGy3XPBhKwMX7jT1_SG-Sb_4ZWZ1I0Ctd-0vNhtmH4gFKaLsV5jhz4vSjYlQ9NR_fXF_b6/pub?output=xlsx'

In [5]:
with pd.ExcelFile(DOCS_LINK) as excel_file:
        nz_df = excel_file.parse('From Briefings', index_col='Date', parse_dates=True, header=1)
        aus_df = excel_file.parse('Australia', index_col='date', parse_dates=True)
        can_df = excel_file.parse('Canada', index_col='date', parse_dates=True)
        uk_df = excel_file.parse('UK', index_col='date', parse_dates=True)
nz_df = nz_df[~nz_df.isna().all(1)]
us_df = pd.read_csv('https://covidtracking.com/api/us/daily.csv', parse_dates=True, index_col=0)


In [6]:
us_df.sort_index()['positive'].diff()

date
2020-03-04        NaN
2020-03-05       58.0
2020-03-06       47.0
2020-03-07      118.0
2020-03-08       76.0
2020-03-09      167.0
2020-03-10      194.0
2020-03-11      276.0
2020-03-12      261.0
2020-03-13      607.0
2020-03-14      528.0
2020-03-15      723.0
2020-03-16      846.0
2020-03-17     1703.0
2020-03-18     2008.0
2020-03-19     3989.0
2020-03-20     5314.0
2020-03-21     6164.0
2020-03-22     8682.0
2020-03-23    10273.0
2020-03-24     9802.0
2020-03-25    11974.0
2020-03-26    16807.0
2020-03-27    18678.0
2020-03-28    18821.0
2020-03-29    20827.0
2020-03-30    21469.0
2020-03-31    24153.0
2020-04-01    26133.0
2020-04-02    28283.0
2020-04-03    32889.0
2020-04-04    33767.0
Name: positive, dtype: float64

In [7]:
def make_country_tests_df(df, columns, millions, new_names=['confirmed', 'tests']):
    new_df = df[columns].rename(columns=dict(zip(columns,new_names))).sort_index()
    for n in new_names:
        col = new_df[n]
        new_df[n+'/1M']=col/millions
        new_df['new '+n] = col.diff()
    return new_df

In [8]:
MILLIONS = {'NZ':4.974490, 'NZC':4.974490, 'US':329.544974, 'Aus':25.663280, 'Can': 37.976754, 'UK':66.435600}
FULL_NAMES = {'NZ':'New Zealand', 'NZC':'NZ (positives tests)', 'Aus': 'Australia', 'Can': 'Canada', 'UK':'United Kingdom', 'US':'USA'}
COLUMN_NAMES={'NZ':['Cases', 'Tests'], 'NZC':['Confirmed', 'Tests'], 'US':['positive', 'posNeg'], 'Can':['positives', 'tests'], 'Aus': ['cases', 'tests'], 'UK': ['cases', 'tests']}
def make_tests_df(dfs_map):
    tests_df = pd.concat({FULL_NAMES.get(c, c):make_country_tests_df(df, COLUMN_NAMES[c], MILLIONS[c]) for c,df in dfs_map.items()}, sort=True, names=['country']).reset_index()
    tests_df['case rate'] = tests_df['confirmed']/tests_df['tests']
    tests_df['daily rate'] = tests_df['new confirmed']/tests_df['new tests']
    return tests_df
    
    

In [9]:
tests_df = make_tests_df({'NZ':nz_df, 'NZC':nz_df, 'Aus':aus_df, 'Can':can_df, 'UK':uk_df, 'US':us_df })
tests_df['nice_date']=tests_df.iloc[:,1].dt.strftime('%e %b %Y')

In [10]:
now = pd.Timestamp('now', tz='Pacific/Auckland').strftime('%e %b %Y %H:%M (%z)')

In [11]:
LABELS = {
    'case rate':'confirmed<br>case rate', 
    'tests/1M':'total tests per 1M population (10k is 1% of population)', 
    'confirmed/1M':'confirmed cases<br>per 1M population',
    'title': 'Comparison of confirmed Covid-19 cases and total tests for selected countries normalised by population<br>Last update: '+now,
}
HOVER_TEMPLATE = hovertemplate='<br>'.join([
    '<b>%{fullData.name}</b> - %{hovertext}',
    'confirmed cases=%{customdata[0]}<br>  - up %{customdata[2]} to %{y:.1f}/1M',
    'total tests=%{customdata[1]}<br>  - up %{customdata[3]} to %{x:.1f}/1M',
    'confirmed case rate=%{marker.color:.1%}<br>  - 1 day rate=%{customdata[4]:.1%}'])
COLORWAY = plotly.io.templates['plotly_white']['layout']['colorway']
UPDATES = {
    'traces': dict(line_width=4, marker_size=8, mode='markers+lines'),
    'layout': dict(coloraxis=dict(colorbar=dict(thickness=20, tickformat='.0%'), colorscale=px.colors.sequential.Plasma[2:-2]), 
                   legend=dict(bgcolor='#eeeeee', x=.025, y=0.95), ), 
    'nzc_trace': dict(selector={'name': FULL_NAMES['NZC']}, marker_size=6, line=dict(dash='dot', width=3)),
}

In [12]:
fig=(px.scatter(tests_df, y='confirmed/1M', x='tests/1M',symbol='country', color='case rate', symbol_map={FULL_NAMES['NZC']:'diamond'}, range_color=[0,.2], hover_data=['confirmed', 'tests', 'new confirmed', 'new tests', 'daily rate'], hover_name='nice_date', labels=LABELS, title=LABELS['title'])
    .update_traces(hovertemplate=HOVER_TEMPLATE, **UPDATES['traces'])
    .update_layout(**UPDATES['layout'])
    .update_traces(**UPDATES['nzc_trace']) )
for cw, trace in zip(COLORWAY[:3], fig.data[1:]):
    trace.update(line_color=cw)
fig

In [13]:
fig.write_html('../Coronavirus_testing_comparison.html', include_plotlyjs='cdn')

In [14]:
#px.colors.sequential.swatches()